In [1]:
% matplotlib inline

from scikits.talkbox.features import mfcc
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import soundfile as sf
import pyaudio as pa
import os

random_state=24

音声ファイルimport

In [2]:
files = []
datapath = '../cry_data/devided_wav_data/'
# datapath = 'devided_wav_data2/'

for x in os.listdir(datapath):  
    if '.wav' in x:
        files.append(x)

各クラスのデータ数をそろえる

In [59]:
hu_count = 0
ti_count = 0
improved_files = []
for f in files:
    if '-hu.' in f:
        if hu_count < 18:
            hu_count += 1
            improved_files.append(f)
    elif '-ti.' in f:
        if ti_count < 18:
            ti_count += 1
            improved_files.append(f)
    else:
        improved_files.append(f)

del files
files = improved_files

mfccで特徴量とラベルのセット作成

In [3]:
label_dict = {'hu':0, 'ti':1, 'dc':2}

X = []
y = []
x_seq = []
for f in files:
    x, sample_rate = sf.read(datapath + f)
    x_seq.append(x)
    x = np.clip(x, 1e-10, 1)
    ceps,mspec,spec = mfcc(x, nwin=256, nfft=512, fs=8000, nceps=13)
    X.append(np.mean(ceps, axis=0))
    if '-hu.' in f:
        y.append(label_dict['hu'])
    elif '-ti.' in f:
        y.append(label_dict['ti'])
    else:
        y.append(label_dict['dc'])

X = np.array(X)
y = np.array(y)

welchメソッドによるパワースペクトルで特徴量とラベルのセット作成

In [2]:
import scipy.signal

label_dict = {'hu':0, 'ti':1, 'dc':2}

X = []
y = []

fs = 8000
nperseg = 128
nfft=1024
for f in files:
    x, sample_rate = sf.read(datapath + f)
    freq, P = scipy.signal.welch(x, fs, window='hamming', nperseg=nperseg, nfft=nfft)
    X.append(P)
    if '-hu.' in f:
        y.append(label_dict['hu'])
    elif '-ti.' in f:
        y.append(label_dict['ti'])
    else:
        y.append(label_dict['dc'])

X = np.array(X)
y = np.array(y)

NameError: name 'files' is not defined

In [150]:
# 圧縮
from sklearn.decomposition import PCA
pca = PCA(n_components=13, random_state=random_state)
X = pca.fit_transform(X)

分類器で学習、評価

In [92]:
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.svm import LinearSVC, SVC
from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.neural_network import MLPClassifier

precision = 0
recall = 0
f_value = 0

n_splits = 10

skf = StratifiedKFold(n_splits=n_splits, random_state=random_state, shuffle=True)
for train_index, test_index in skf.split(X, y):
    X_train, X_valid = X[train_index], X[test_index]
    y_train, y_valid = y[train_index], y[test_index]
    # clf = SVC(random_state=random_state, kernel='poly', probability=True)
    # clf = RandomForestClassifier(n_estimators=498, random_state=random_state)
    # clf = XGBClassifier(max_depth=5, learning_rate=0.06, n_estimators=200, seed=random_state)
    # clf = KNeighborsClassifier(n_neighbors=3, weights='uniform', p=1)
    # clf = QuadraticDiscriminantAnalysis()
    #clf = MLPClassifier(hidden_layer_sizes=(50, 2), activation='tanh', random_state=random_state, alpha=0.01,\
    #                    learning_rate_init=0.01, max_iter=500, beta_1=0.8)
    # clf = KNeighborsClassifier(algorithm='ball_tree',n_neighbors=3, weights='uniform', p=1)
    clf = KNeighborsClassifier(algorithm='auto',n_neighbors=3, weights='uniform', p=1)
    
    clf.fit(X_train, y_train)
    pred = clf.predict(X_valid)
    precision += precision_score(y_valid, pred, average='micro')
    recall += recall_score(y_valid, pred, average='micro')
    f_value += f1_score(y_valid, pred, average='micro')

precision = precision / float(n_splits)
recall = recall / float(n_splits)
f_value = f_value / float(n_splits)

In [93]:
precision

0.59736044657097287

In [91]:
f_value

0.59736044657097287

アンサンブル

In [11]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC, SVC
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from sklearn.ensemble import VotingClassifier

precision = 0
recall = 0
f_value = 0
accuracy = 0

n_splits = 10

skf = StratifiedKFold(n_splits=n_splits, random_state=random_state, shuffle=True)
for train_index, test_index in skf.split(X, y):
    X_train, X_valid = X[train_index], X[test_index]
    y_train, y_valid = y[train_index], y[test_index]
    clf1 = SVC(random_state=random_state, kernel='poly', probability=True)
    clf2 = RandomForestClassifier(n_estimators=498, random_state=random_state)
    clf3 = XGBClassifier(max_depth=5, learning_rate=0.06, n_estimators=200, seed=random_state)
    clf4 = KNeighborsClassifier(n_neighbors=3, weights='uniform', p=1)
    clf5 = QuadraticDiscriminantAnalysis()
    clf6 = MLPClassifier(hidden_layer_sizes=(50, 2), activation='tanh', random_state=random_state, alpha=0.01,\
                        learning_rate_init=0.01, max_iter=500, beta_1=0.8)
    eclf = VotingClassifier(estimators=[('rf', clf2), ('knn', clf4), ('qda', clf5)], voting='hard')
    eclf.fit(X_train, y_train)
    pred = eclf.predict(X_valid)
    precision += precision_score(y_valid, pred, average='macro')
    recall += recall_score(y_valid, pred, average='macro')
    f_value += f1_score(y_valid, pred, average='macro')
    accuracy += accuracy_score(y_valid, pred)

precision = precision / float(n_splits)
recall = recall / float(n_splits)
f_value = f_value / float(n_splits)
accuracy = accuracy / float(n_splits)

In [12]:
accuracy

0.66697767145135578

In [13]:
precision

0.71663419913419912

In [14]:
f_value

0.65120333141230968

In [97]:
from sklearn.externals import joblib
clf2 = RandomForestClassifier(n_estimators=498, random_state=random_state)
clf4 = KNeighborsClassifier(n_neighbors=3, weights='uniform', p=1)
clf5 = QuadraticDiscriminantAnalysis()
eclf = VotingClassifier(estimators=[('rf', clf2), ('knn', clf4), ('qda', clf5)], voting='hard')
eclf.fit(X, y)
joblib.dump(eclf, 'clf_rf_knn_qda.pkl.cmp', compress=True)

['clf_rf_knn_qda.pkl.cmp']